In [1]:
import sys
sys.path.append("../")

In [4]:
from libraries.dataManipulation import *
from libraries.gridManipulation import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import warnings
import matplotlib.colors as colors
import matplotlib.animation as animation
import matplotlib.lines as lines
import datetime
import math
%matplotlib notebook

In [5]:
data = getDay(20220320, 'ehv')

targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220320
Files listed. Accessible as <this-object>.filelist
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220320/EHV_Platform2.1_2022032000_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220320/EHV_Platform2.1_2022032000_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220320/EHV_Platform2.1_2022032001_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220320/EHV_Platform2.1_2022032002_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220320/EHV_Platform2.1_2022032003_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to f

In [6]:
def direction_label(data):
    def compute_direction(group):
        first_appearance_index = group.index[0]
        last_appearance_index = group.index[-1]
        start_coordinate = group.loc[first_appearance_index]['x_pos']
        end_coordinate = group.loc[last_appearance_index]['x_pos']
        x_displacement = end_coordinate - start_coordinate
        if x_displacement > 0:
            return 'onboarding'
        elif x_displacement < 0:
            return 'offboarding'
        else:
            return 'undefined'
    
    result_series = data.groupby('tracked_object').apply(compute_direction)
    result_df = result_series.to_frame()
    result_df.columns = ['direction']
    result_df = result_df.reset_index()
    return result_df


In [7]:
%%time
directions = direction_label(data)

Wall time: 11.9 s


In [8]:
directions

,tracked_object,direction
0,3625345,offboarding
1,3625359,onboarding
2,3625382,offboarding
3,3625409,onboarding
4,3625508,onboarding
...,...,...
38558,3666363,offboarding
38559,3666364,offboarding
38560,3666365,offboarding
38561,3666366,offboarding


In [9]:
percentages = directions['direction'].value_counts(normalize=True) * 100

In [10]:
percentages['offboarding']/0

<ipython-input-10-762a15bb146e>:1: RuntimeWarning: divide by zero encountered in double_scalars
  percentages['offboarding']/0


inf

In [11]:
dh.fetch_background_image_from_path('ehv/background_images/EHV.Perron2.1_multisensor.png')
df = data
directions = directions.set_index('tracked_object')
directions

directions_series = pd.Series(directions.direction)
directions_series

trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/background_images/EHV.Perron2.1_multisensor.png
background fetched. Accessible as <this-object>.bg


tracked_object
3625345    offboarding
3625359     onboarding
3625382    offboarding
3625409     onboarding
3625508     onboarding
              ...     
3666363    offboarding
3666364    offboarding
3666365    offboarding
3666366    offboarding
3666367    offboarding
Name: direction, Length: 38563, dtype: object

In [12]:
df['onboarding_or_offboarding'] = df['tracked_object'].map(directions_series)
df['onboarding_or_offboarding'] = df['tracked_object'].map(directions_series)

df.loc[df['onboarding_or_offboarding']=='onboarding','onboarding_or_offboarding']=1
df.loc[df['onboarding_or_offboarding']=='offboarding','onboarding_or_offboarding']=2

df

,date_time_utc,tracked_object,y_pos,x_pos,onboarding_or_offboarding
0,2022-03-19 23:00:00.000,3625345,-2609.0,62330.0,2
1,2022-03-19 23:00:00.000,3625359,1184.0,14600.0,1
2,2022-03-19 23:00:00.000,3625382,-3057.0,59640.0,2
3,2022-03-19 23:00:00.000,3625409,1463.0,2850.0,1
4,2022-03-19 23:00:00.000,3625508,1804.0,13022.0,1
...,...,...,...,...,...
19974617,2022-03-20 22:59:59.800,3666362,1803.0,63683.0,1
19974618,2022-03-20 22:59:59.900,3666228,-3333.0,60091.0,1
19974619,2022-03-20 22:59:59.900,3666298,2499.0,64864.0,1
19974620,2022-03-20 22:59:59.900,3666307,-4949.0,62837.0,1


In [1]:
xmin = -5134
ymin = -17731
xmax =  75135
ymax =  11645

cmap = colors.LinearSegmentedColormap.from_list("",["blue","blue","red"])#("",["black","black"])
#rect1 = patches.Rectangle((boarding_limits_1[0],boarding_limits_1[2]),boarding_limits_1[1]-boarding_limits_1[0],boarding_limits_1[3]-boarding_limits_1[2],fill=False,color="red",linewidth=2,angle=-0.8)
#rect2 = patches.Rectangle((boarding_limits_2[0],boarding_limits_2[2]),boarding_limits_2[1]-boarding_limits_2[0],boarding_limits_2[3]-boarding_limits_2[2],fill=False,color="red",linewidth=2,angle=-0.8)





class AnimatedScatter:
    
    def __init__(self):
        
        self.fig, self.ax = plt.subplots()
        self.ani = animation.FuncAnimation(self.fig, self.update, init_func = self.setup_plot, interval = 100, blit=False)
        self.date_time = datetime.datetime(year=2022,month=3,day=20,hour=9,minute=3)
        
    def setup_plot(self):
        print(1)
        # 2022 september 28th: 18:40, 19:06, 17:38, 17:50
        temp_df = df[df['date_time_utc'] == self.date_time].copy()
        x = temp_df['x_pos']
        y = temp_df['y_pos']
        x.head()
        on_or_off = temp_df['onboarding_or_offboarding']
        self.scat = self.ax.scatter(x,y,c=on_or_off,cmap=cmap,s=5)
        #plt.imshow(dh.bg)
        self.ax.axis([xmin,xmax,ymin,ymax])
        self.txt = plt.text(-200,6000, self.date_time, c='white')
        
        
        #plt.legend((line1,line2),('offboarding','onboarding'),numpoints=1,loc=1)
        
        #plt.gca().add_patch(rect1)
        #plt.gca().add_patch(rect2)
        print(2)
        return self.scat,

    def update(self, i=0):
        
        self.date_time =  self.date_time + datetime.timedelta(milliseconds = 500) # we can change timedelta, should be multiple of 100
        temp_df = df[df['date_time_utc'] ==  self.date_time].copy()
        x = temp_df['x_pos']
        y = temp_df['y_pos']
        on_or_off = temp_df['onboarding_or_offboarding']
        self.scat.set_offsets(np.c_[x,y])
        print(on_or_off)
        self.scat.set_array(on_or_off)
        self.txt.set_text( self.date_time)
        return self.scat,

a = AnimatedScatter()

plt.imshow(dh.bg,zorder = 0, extent=[xmin, xmax, ymin, ymax])
plt.show()
a.setup_plot()
a.update()
"""
date_time = datetime.datetime(year=2022,month=3,day=20,hour=9,minute=3)
temp_df = df[df['date_time_utc'] == date_time].copy()
x = temp_df['x_pos']
y = temp_df['y_pos']
x.head()
"""










NameError: name 'colors' is not defined